# Библиотеки Python для DS (семинары)

Урок 8. На практике использование методов генерации признаков

В вашем распоряжении набор данных о качестве сна Sleep_health_and_lifestyle_dataset.csv.
Ваша задача — подготовить данные к моделированию, сгенерировать новые признаки, применить методы отбора признаков для улучшения качества предсказательной модели и подобрать оптимальные гиперпараметры для модели регрессии.

1. Предобработка данных:
— Очистите данные от пропусков и аномалий.
— Преобразуйте категориальные переменные с помощью One-Hot Encoding или Label Encoding.

2. Генерация новых признаков:
— Создайте комбинированные признаки, которые могут помочь улучшить модель.
— Используйте полиномиальные признаки для числовых переменных.

3. Отбор признаков:
— Примените несколько методов отбора признаков (например, RFE, SelectKBest).
— Сравните качество модели до и после отбора признаков.

4. Подбор гиперпараметров:
— Используйте GridSearchCV или RandomizedSearchCV для настройки параметров вашей модели регрессии.
— Оцените, как изменение гиперпараметров влияет на качество предсказаний.

## Выполнение домашней работы

##### Используем предоставленый к заданию файл fetch_california_housing.xlsx

In [1]:
# Библиотеки для работы с датасетом и графиками
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# Загружаем датасет
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')

##### 1. Предобработка данных:

— Очистите данные от пропусков и аномалий.

— Преобразуйте категориальные переменные с помощью One-Hot Encoding или Label Encoding.

In [3]:
# Выводим дата фрейм
df

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea


Этот фрейм данных содержит следующие столбцы:

1. Person ID (Идентификатор для каждого человека)
2. Gender (Пол человека (мужчина/женщина))
3. Age (Возраст человека в годах)
4. Occupation (Род занятий или профессия человека)
5. Sleep Duration (hours) (Количество часов, которые человек спит в сутки)
6. Quality of Sleep (scale: 1-10) (Субъективная оценка качества сна в диапазоне от 1 до 10)
7. Physical Activity Level (minutes/day) (Количество минут, которые человек ежедневно уделяет физической активности)
8. Stress Level (scale: 1-10) (Субъективная оценка уровня стресса, испытываемого человеком, в диапазоне от 1 до 10)
9. BMI Category (Категория ИМТ человека (например, недостаточный вес, нормальный, избыточный вес с избыточной массой тела))
10. Blood Pressure (systolic/diastolic) (Измерение артериального давления человека, показывающее превышение систолического давления над диастолическим давлением)
11. Heart Rate (bpm) (Частота сердечных сокращений человека в состоянии покоя в ударах в минуту)
12. Daily Steps (Количество шагов, которые человек делает за день)
13. Sleep Disorder (Наличие или отсутствие у человека нарушений сна (отсутствие таковых, бессонница, апноэ во сне))

In [4]:
# Выводим общую информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


Из общей информации следует:
1. Всего 13 колонок, DataFrame содержит 374 записи, от 0 до 373
2. Нулевые значения отсуствуют
3. Есть пропущеные записи в колонке Sleep Disorder
4. Колонки: Person ID, Age, Quality of Sleep, Physical Activity Level, Stress Level, Heart Rate, Daily Steps находятся в целочисленых значениях int64
5. Колонки: Sleep Duration находится в дробном формате float64
6. Колонки: Gender, Occupation, BMI Category, Blood Pressure, Sleep Disorder являются объектами object

In [5]:
# Статистический анализ числовых переменных
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Person ID,374.0,187.500000,108.108742,1.0,94.25,187.5,280.75,374.0
Age,374.0,42.184492,8.673133,27.0,35.25,43.0,50.00,59.0
Sleep Duration,374.0,7.132086,0.795657,5.8,6.40,7.2,7.80,8.5
Quality of Sleep,374.0,7.312834,1.196956,4.0,6.00,7.0,8.00,9.0
Physical Activity Level,374.0,59.171123,20.830804,30.0,45.00,60.0,75.00,90.0
Stress Level,374.0,5.385027,1.774526,3.0,4.00,5.0,7.00,8.0
Heart Rate,374.0,70.165775,4.135676,65.0,68.00,70.0,72.00,86.0
Daily Steps,374.0,6816.844920,1617.915679,3000.0,5600.00,7000.0,8000.00,10000.0


На основе представленных статистических данных неприрывных признаков можно сделать следующие выводы:

1. Age (Возраст человека в годах): 
- Средний возраст человека на уровне 43.00
- Миниманая 27.00
- Максимальная 59.00
- Медиана равна 42.18
- Отклонение состовляет 8.67
2. Sleep Duration (hours) (Количество часов, которые человек спит в сутки):
- Средние количество часов которое человек спит в сутки на уровне 7.20
- Миниманая 5.80
- Максимальная 8.50
- Медиана равна 7.13
- Отклонение состовляет 0.79
3. Quality of Sleep (scale: 1-10) (Субъективная оценка качества сна в диапазоне от 1 до 10):
- Средняя оценка качества сна на уровне 7.00
- Миниманая 4.00
- Максимальная 9.00
- Медиана равна 7.31
- Отклонение состовляет 1.19
4. Physical Activity Level (minutes/day) (Количество минут, которые человек ежедневно уделяет физической активности):
- Средние количество минут которые человек ежедневно уделяет физической активности на уровне 60.00
- Миниманая 30.00
- Максимальная 90.00
- Медиана равна 59.17
- Отклонение состовляет 20.83
5. Stress Level (scale: 1-10) (Субъективная оценка уровня стресса, испытываемого человеком, в диапазоне от 1 до 10):
- Средние субъективная оценка уровня стреса на уровне 5.00
- Миниманая 3.00
- Максимальная 8.00
- Медиана равна 5.38
- Отклонение состовляет 5.38
6. Heart Rate (bpm) (Частота сердечных сокращений человека в состоянии покоя в ударах в минуту):
- Средняя частота сердечных сокращений на уровне 70.00
- Миниманая 65.00
- Максимальная 86.00
- Медиана равна 70.16
- Отклонение состовляет 4.13
7. Daily Steps (Количество шагов, которые человек делает за день):
- Средние количество шагов на уровне 7000.00
- Миниманая 3000.00
- Максимальная 10000.00
- Медиана равна 6816.84
- Отклонение состовляет 1617.91

In [6]:
# Статистический анализ объектных признаков
df.describe(include='object').T

,count,unique,top,freq
Gender,374,2,Male,189
Occupation,374,11,Nurse,73
BMI Category,374,4,Normal,195
Blood Pressure,374,25,130/85,99
Sleep Disorder,155,2,Sleep Apnea,78


1. Gender (Пол человека (мужчина/женщина)):
- В данных присутствуют записи о поле у 374 индивидов.
- Имеются всего две уникальные категории пола.
- Наиболее часто встречающийся пол - Мужчина (Male), с частотой 189 раз.

2. Occupation (Род занятий или профессия человека)
- Для 374 индивидов имеются данные о профессии.
- В данных присутствует 11 уникальных профессий.
- Профессия 'Nurse' (Медсестра/Медбрат) встречается чаще всего - 73 раза.

3. BMI Category (Категория ИМТ человека (например, недостаточный вес, нормальный, избыточный вес с избыточной массой тела))
- Информация о категории индекса массы тела (ИМТ) присутствует для всех 374 индивидов.
- Обнаружено 4 уникальные категории ИМТ.
- Наиболее распространенная категория ИМТ - 'Normal' (Норма), встречается 195 раз.

4. Blood Pressure (systolic/diastolic) (Измерение артериального давления человека, показывающее превышение систолического давления над диастолическим давлением)
- Записи об артериальном давлении имеются для каждого из 374 индивидов.
- Выявлено 25 различных значений артериального давления.
- Самый распространенный уровень артериального давления среди участников - 130/85, зафиксирован 99 раз.

5. Sleep Disorder (Наличие или отсутствие у человека нарушений сна (отсутствие таковых, бессонница, апноэ во сне))
- Информация о соновых расстройствах есть для всех 374 индивидов.
- В данных есть 3 уникальных состояния соновых расстройств.
- Наиболее частая запись - 'No Disorder' (Нет расстройства), она встречается в 219 случаях


In [7]:
# Проверяем на дубликаты
print("Количество дубликатов:")
print(df.duplicated().sum())

Количество дубликатов:
0


Дубликаты отсутствуют.

In [8]:
# Смотрим ещё раз тип данных
df.dtypes

Person ID                    int64
Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Quality of Sleep             int64
Physical Activity Level      int64
Stress Level                 int64
BMI Category                object
Blood Pressure              object
Heart Rate                   int64
Daily Steps                  int64
Sleep Disorder              object
dtype: object

In [9]:
# Проверка на нулевые значения
df.isnull().sum()

Person ID                    0
Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Blood Pressure               0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64

In [10]:
# Проверка на NaN значения
df.isnull().any().any()

True

По итогу проверок на NaN значения, пропущеные значения находим 219 пропущеных NaN значений.

In [11]:
# Заменяем пропущенные (NaN) значения.
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('No Disorder')

In [12]:
# Проверка на NaN значения
df.isnull().any().any()

False

Получаем отсуствие пропущеных NaN значений

In [13]:
df['Gender'].value_counts()

Gender
Male      189
Female    185
Name: count, dtype: int64

In [14]:
df['BMI Category'].value_counts()

BMI Category
Normal           195
Overweight       148
Normal Weight     21
Obese             10
Name: count, dtype: int64

In [15]:
df['Occupation'].value_counts()

Occupation
Nurse                   73
Doctor                  71
Engineer                63
Lawyer                  47
Teacher                 40
Accountant              37
Salesperson             32
Software Engineer        4
Scientist                4
Sales Representative     2
Manager                  1
Name: count, dtype: int64

In [16]:
df['Blood Pressure'].value_counts()

Blood Pressure
130/85    99
140/95    65
125/80    65
120/80    45
115/75    32
135/90    27
140/90     4
125/82     4
132/87     3
128/85     3
126/83     2
115/78     2
139/91     2
142/92     2
119/77     2
135/88     2
129/84     2
128/84     2
131/86     2
117/76     2
130/86     2
118/75     2
121/79     1
122/80     1
118/76     1
Name: count, dtype: int64

In [17]:
df['Sleep Disorder'].value_counts()

Sleep Disorder
No Disorder    219
Sleep Apnea     78
Insomnia        77
Name: count, dtype: int64

In [18]:
# Создание экземпляра LabelEncoder
label_encoder = LabelEncoder()

# Создание копии оригинального DataFrame
df_encoder = df.copy()

# Применение Label Encoding к каждому категориальному столбцу
df_encoder['Gender'] = label_encoder.fit_transform(df_encoder['Gender'])
df_encoder['BMI Category'] = label_encoder.fit_transform(df_encoder['BMI Category'])


В виду того, что автоматически LabelEncoder кодирует столбец Sleep Disorder не так как нам нужно, решил задать вручную.

In [19]:
sleep_disorder_mapping = {'No Disorder': 0, 'Sleep Apnea': 2, 'Insomnia': 1} 
df_encoder['Sleep Disorder'] = df_encoder['Sleep Disorder'].map(sleep_disorder_mapping)

In [20]:
df_encoder['BMI Category'].value_counts()

BMI Category
0    195
3    148
1     21
2     10
Name: count, dtype: int64

In [21]:
# Проверяем что всё скодировалось как нужно
df_encoder['Sleep Disorder'].value_counts()

Sleep Disorder
0    219
2     78
1     77
Name: count, dtype: int64

In [22]:
print("\nDataFrame после Label Encoding:")
df_encoder


DataFrame после Label Encoding:


,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,1,27,Software Engineer,6.1,6,42,6,3,126/83,77,4200,0
1,2,1,28,Doctor,6.2,6,60,8,0,125/80,75,10000,0
2,3,1,28,Doctor,6.2,6,60,8,0,125/80,75,10000,0
3,4,1,28,Sales Representative,5.9,4,30,8,2,140/90,85,3000,2
4,5,1,28,Sales Representative,5.9,4,30,8,2,140/90,85,3000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,0,59,Nurse,8.1,9,75,3,3,140/95,68,7000,2
370,371,0,59,Nurse,8.0,9,75,3,3,140/95,68,7000,2
371,372,0,59,Nurse,8.1,9,75,3,3,140/95,68,7000,2
372,373,0,59,Nurse,8.1,9,75,3,3,140/95,68,7000,2


In [23]:
# Создание экземпляра TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование документов в TF-IDF векторы
tfidf_matrix = vectorizer.fit_transform(df['Occupation'])
# Просмотр результатов
print(tfidf_matrix)

# Для получения словаря терминов
print(vectorizer.get_feature_names_out())

  (0, 10)	0.8911405202917568
  (0, 2)	0.45372742158055335
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 7)	0.7071067811865476
  (3, 6)	0.7071067811865476
  (4, 7)	0.7071067811865476
  (4, 6)	0.7071067811865476
  (5, 10)	0.8911405202917568
  (5, 2)	0.45372742158055335
  (6, 11)	1.0
  (7, 1)	1.0
  (8, 1)	1.0
  (9, 1)	1.0
  (10, 1)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 1)	1.0
  (14, 1)	1.0
  (15, 1)	1.0
  (16, 5)	1.0
  (17, 1)	1.0
  (18, 5)	1.0
  (19, 1)	1.0
  (20, 1)	1.0
  :	:
  (349, 5)	1.0
  (350, 5)	1.0
  (351, 5)	1.0
  (352, 5)	1.0
  (353, 5)	1.0
  (354, 5)	1.0
  (355, 5)	1.0
  (356, 5)	1.0
  (357, 5)	1.0
  (358, 5)	1.0
  (359, 5)	1.0
  (360, 5)	1.0
  (361, 5)	1.0
  (362, 5)	1.0
  (363, 5)	1.0
  (364, 5)	1.0
  (365, 5)	1.0
  (366, 5)	1.0
  (367, 5)	1.0
  (368, 5)	1.0
  (369, 5)	1.0
  (370, 5)	1.0
  (371, 5)	1.0
  (372, 5)	1.0
  (373, 5)	1.0
['accountant' 'doctor' 'engineer' 'lawyer' 'manager' 'nurse'
 'representative' 'sales' 'salesperson' 'scientist' 'software' 'teacher']


In [24]:
# Создание DataFrame из матрицы TF-IDF
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Конкатенируем исходный DataFrame с новым DataFrame векторов TF-IDF
df_encoder = pd.concat([df_encoder, tfidf_df], axis=1).drop('Occupation', axis=1)

In [25]:
df_encoder

,Person ID,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,...,engineer,lawyer,manager,nurse,representative,sales,salesperson,scientist,software,teacher
0,1,1,27,6.1,6,42,6,3,126/83,77,...,0.453727,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.891141,0.0
1,2,1,28,6.2,6,60,8,0,125/80,75,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
2,3,1,28,6.2,6,60,8,0,125/80,75,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,4,1,28,5.9,4,30,8,2,140/90,85,...,0.000000,0.0,0.0,0.0,0.707107,0.707107,0.0,0.0,0.000000,0.0
4,5,1,28,5.9,4,30,8,2,140/90,85,...,0.000000,0.0,0.0,0.0,0.707107,0.707107,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,0,59,8.1,9,75,3,3,140/95,68,...,0.000000,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
370,371,0,59,8.0,9,75,3,3,140/95,68,...,0.000000,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
371,372,0,59,8.1,9,75,3,3,140/95,68,...,0.000000,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
372,373,0,59,8.1,9,75,3,3,140/95,68,...,0.000000,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.000000,0.0


##### 2. Генерация новых признаков:

— Создайте комбинированные признаки, которые могут помочь улучшить модель.

— Используйте полиномиальные признаки для числовых переменных.

##### 3. Отбор признаков:

— Примените несколько методов отбора признаков (например, RFE, SelectKBest).

— Сравните качество модели до и после отбора признаков.

##### 4. Подбор гиперпараметров:

— Используйте GridSearchCV или RandomizedSearchCV для настройки параметров вашей модели регрессии.

— Оцените, как изменение гиперпараметров влияет на качество предсказаний.